In [2]:
import pandas as pd
import psycopg2
from datetime import datetime, time, timedelta
 

In [3]:
HOST = "rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net"
DB = "db1"
USER = "test_user"
PASSWORD = "j2M{CnnFq@"
PORT = "6432"

In [4]:
try:
    conn = psycopg2.connect(
        host=HOST,
        port=PORT,
        database=DB,
        user=USER,
        password=PASSWORD
    )
    print("Connection successful")
except Exception as e:
    print(f"Error: {e}")   

Connection successful


In [5]:
managers_query = "SELECT * FROM test.managers"
chat_messages_query = "SELECT * FROM test.chat_messages"
rops_query = "SELECT * FROM test.rops"

managers = pd.read_sql_query(managers_query, conn)
chat_messages = pd.read_sql_query(chat_messages_query, conn)
rops = pd.read_sql_query(rops_query, conn)
conn.close()

C:\Users\user\AppData\Local\Temp\ipykernel_6604\1375247572.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  managers = pd.read_sql_query(managers_query, conn)
C:\Users\user\AppData\Local\Temp\ipykernel_6604\1375247572.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chat_messages = pd.read_sql_query(chat_messages_query, conn)
C:\Users\user\AppData\Local\Temp\ipykernel_6604\1375247572.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rops = pd.read_sql_query(rops_query, conn)


In [8]:
chat_messages['created'] = pd.to_datetime(chat_messages['created_at'], unit='s')

In [10]:
def change_time(created):
    if created.hour < 9 or (created.hour == 9 and created.minute < 30):
        return created.replace(hour=9, minute=30, second=0, microsecond=0)
    else:
        return created
chat_messages['created'] = chat_messages['created'].apply(change_time)

In [11]:
chat_messages.head()

,message_id,type,entity_id,created_by,created_at,created
0,"""01jb7da570sf4f65xdf0ptvv9q""",incoming_chat_message,37556493,0,1730046924,2024-10-27 16:35:24
1,"""01jb3t6bc8gvyfnd063nd91ng4""",incoming_chat_message,37549491,0,1729926213,2024-10-26 09:30:00
2,"""01jb6gj9ngwk0ybbmg9w90pbqg""",incoming_chat_message,37531455,0,1730016782,2024-10-27 09:30:00
3,"""01jb75bkprk0d7hht1g5vy1pm4""",incoming_chat_message,37553371,0,1730038583,2024-10-27 14:16:23
4,"""01jb3t5xprbph1c6pym8ervxpm""",incoming_chat_message,37548675,0,1729926199,2024-10-26 09:30:00


In [23]:
chat_messages = chat_messages.sort_values(by=['entity_id', 'created'])
chat_messages['group'] = ((chat_messages['entity_id'] != chat_messages.shift()['entity_id']) | (chat_messages['type'] != chat_messages.shift()['type'])).cumsum()

In [29]:
chat_messages = chat_messages.groupby(by='group').first().reset_index()

In [30]:
chat_messages

,group,message_id,type,entity_id,created_by,created_at,created
0,1,"""01jb4pjb1g22ts7z6arcjv11w6""",incoming_chat_message,35810509,0,1729955966,2024-10-26 15:19:26
1,2,"""01jb4pncn7gm1re1zdxkzhnp6s""",outgoing_chat_message,35810509,10262493,1729956065,2024-10-26 15:21:05
2,3,"""01jb4qedfgxnd2yqr4w26hs4q5""",incoming_chat_message,35810509,0,1729956886,2024-10-26 15:34:46
3,4,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,1729957006,2024-10-26 15:36:46
4,5,"""01jb7gxwb0sepv9vtekrgj3v6d""",incoming_chat_message,35945535,0,1730050716,2024-10-27 17:38:36
...,...,...,...,...,...,...,...
17818,17819,"""01jb7w5npw67ghth0ctgqkvrzr""",outgoing_chat_message,37557215,6744792,1730062505,2024-10-27 20:55:05
17819,17820,"""01jb7vxxz0djjdc4qjezwhycev""",incoming_chat_message,37557225,0,1730062252,2024-10-27 20:50:52
17820,17821,"""01jb7w3wcr7w96me3k4t5y9a3w""",incoming_chat_message,37557229,0,1730062447,2024-10-27 20:54:07
17821,17822,"""01jb7we9ap70azqhbmqsvxqafh""",outgoing_chat_message,37557233,6780177,1730062787,2024-10-27 20:59:47


In [31]:
chat_messages['time_diff'] = chat_messages.groupby('entity_id')['created'].diff() 
chat_messages

,group,message_id,type,entity_id,created_by,created_at,created,time_diff
0,1,"""01jb4pjb1g22ts7z6arcjv11w6""",incoming_chat_message,35810509,0,1729955966,2024-10-26 15:19:26,NaT
1,2,"""01jb4pncn7gm1re1zdxkzhnp6s""",outgoing_chat_message,35810509,10262493,1729956065,2024-10-26 15:21:05,0 days 00:01:39
2,3,"""01jb4qedfgxnd2yqr4w26hs4q5""",incoming_chat_message,35810509,0,1729956886,2024-10-26 15:34:46,0 days 00:13:41
3,4,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,1729957006,2024-10-26 15:36:46,0 days 00:02:00
4,5,"""01jb7gxwb0sepv9vtekrgj3v6d""",incoming_chat_message,35945535,0,1730050716,2024-10-27 17:38:36,NaT
...,...,...,...,...,...,...,...,...
17818,17819,"""01jb7w5npw67ghth0ctgqkvrzr""",outgoing_chat_message,37557215,6744792,1730062505,2024-10-27 20:55:05,0 days 00:11:37
17819,17820,"""01jb7vxxz0djjdc4qjezwhycev""",incoming_chat_message,37557225,0,1730062252,2024-10-27 20:50:52,NaT
17820,17821,"""01jb7w3wcr7w96me3k4t5y9a3w""",incoming_chat_message,37557229,0,1730062447,2024-10-27 20:54:07,NaT
17821,17822,"""01jb7we9ap70azqhbmqsvxqafh""",outgoing_chat_message,37557233,6780177,1730062787,2024-10-27 20:59:47,NaT
